In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/h2oai-predict-the-llm/sample_submission.csv
/kaggle/input/h2oai-predict-the-llm/train.csv
/kaggle/input/h2oai-predict-the-llm/test.csv


In [36]:

! pip install datasets transformers

^C
ERROR: Operation cancelled by user


In [37]:
train = "/kaggle/input/h2oai-predict-the-llm/train.csv"
test = '/kaggle/input/h2oai-predict-the-llm/test.csv'

In [38]:
train_df = pd.read_csv(train)
train_df

,Question,Response,target
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3
...,...,...,...
3971,You will always find me in the past. I can be ...,Please select one of the options from the tabl...,0
3972,To give me to someone i don’t belong to is cow...,________________________________. (20 points),0
3973,Could severe abdominal pain and vomiting be in...,Abdominal pain and vomiting can indeed be symp...,1
3974,How can I increase my productivity at work or ...,1. Prioritize tasks: Make a to-do list with th...,1


In [39]:
test_df = pd.read_csv(test)
test_df

,id,Question,Response
0,0,Multiply 13 by 7.,Multiplying 13 by 7 gives a result of 64.\nThe...
1,1,Who were the main protagonists in the Hundred ...,The most important characters in this war incl...
2,2,What was the importance of the Truman Doctrine?,"The Truman Doctrine, also known as the United ..."
3,3,Which film features a character named Jack Ske...,The Nightmare Before Christmas
4,4,What are the components of the cytoskeleton?,"The cytoskeleton, which provides the framework..."
...,...,...,...
996,996,Which film features a character named Jack Ske...,Jack Skellington is a main character in the Pu...
997,997,How does the size and scale of your model affe...,The size and scale of a language model can hav...
998,998,Can frequent urination and pelvic pain be symp...,"Yes, frequent urination and pelvic pain can be..."
999,999,Could sudden and severe chest pain be indicati...,"Yes, sudden and severe chest pain can be a sym..."


In [40]:
train_df

,Question,Response,target
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3
...,...,...,...
3971,You will always find me in the past. I can be ...,Please select one of the options from the tabl...,0
3972,To give me to someone i don’t belong to is cow...,________________________________. (20 points),0
3973,Could severe abdominal pain and vomiting be in...,Abdominal pain and vomiting can indeed be symp...,1
3974,How can I increase my productivity at work or ...,1. Prioritize tasks: Make a to-do list with th...,1


In [41]:
train_df.isna().sum() , test_df.isna().sum()

(Question    0
 Response    7
 target      0
 dtype: int64,
 id          0
 Question    0
 Response    2
 dtype: int64)

In [42]:
train_df.fillna(" ", inplace=True)
test_df.fillna(" ", inplace=True)

In [43]:
train_df.isna().sum() , test_df.isna().sum()

(Question    0
 Response    0
 target      0
 dtype: int64,
 id          0
 Question    0
 Response    0
 dtype: int64)

In [44]:
from datasets import Dataset
dataset_1 = Dataset.from_pandas(train_df)
dataset_2 = Dataset.from_pandas(test_df)

In [45]:
dataset_1 , dataset_2

(Dataset({
     features: ['Question', 'Response', 'target'],
     num_rows: 3976
 }),
 Dataset({
     features: ['id', 'Question', 'Response'],
     num_rows: 1001
 }))

In [46]:
new_dataset= dataset_1.train_test_split(test_size=0.1)

In [47]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Response', 'target'],
        num_rows: 3578
    })
    test: Dataset({
        features: ['Question', 'Response', 'target'],
        num_rows: 398
    })
})

In [48]:
from datasets import DatasetDict

In [49]:
hf_dataset = DatasetDict({
    "train": new_dataset['train'],
    'valid': new_dataset['test'],
})

In [50]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Response', 'target'],
        num_rows: 3578
    })
    valid: Dataset({
        features: ['Question', 'Response', 'target'],
        num_rows: 398
    })
})

In [51]:
# Combine 'Question' and 'Response' into a single string
def combine_question_response(example):
    combined_text = f"{example['Question']} {example['Response']}"
    return {'combined_text': combined_text}

# Apply the function to your entire dataset
dataset = hf_dataset.map(combine_question_response)



  0%|          | 0/3578 [00:00<?, ?ex/s]

  0%|          | 0/398 [00:00<?, ?ex/s]

In [52]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Response', 'target', 'combined_text'],
        num_rows: 3578
    })
    valid: Dataset({
        features: ['Question', 'Response', 'target', 'combined_text'],
        num_rows: 398
    })
})

In [53]:
checkpoint = 'bert-base-uncased'

In [54]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [56]:
def tokenize_function(examples):
    return tokenizer(examples['combined_text'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [57]:
tokenized_dataset = tokenized_dataset.remove_columns(['Question','Response','combined_text'])

In [58]:
tokenized_dataset = tokenized_dataset.rename_column('target',"labels")

tokenized_dataset.set_format('torch')


In [59]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3578
    })
    valid: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 398
    })
})

In [60]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
batch_size = 8
metric_name = "f1"

In [62]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

In [63]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [64]:
from transformers import TrainingArguments, Trainer


In [65]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    evaluation_strategy='steps',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='/kaggle/working/',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

In [66]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [73]:
from sklearn.metrics import hamming_loss

# Assuming y_true and y_pred are your true labels and predicted labels
hamming_loss_score = hamming_loss(y_true, y_pred)
print("Hamming Loss:", hamming_loss_score)


NameError: name 'y_true' is not defined

In [67]:
outputs = model(input_ids=tokenized_dataset['train']['input_ids'][0].unsqueeze(0), labels=tokenized_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(1.7450, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.1686,  0.1371, -0.6819, -0.0440,  0.0252, -0.0009, -0.0417]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [68]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [72]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets

In [ ]:
trainer.evaluate()